In [628]:
from google.cloud import bigquery
import pandas as pd

In [629]:
%matplotlib inline
import matplotlib.pyplot as plt

In [630]:
client = bigquery.Client()

/home/thorad/.conda/envs/axiom/lib/python3.6/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/home/thorad/.conda/envs/axiom/lib/python3.6/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.

In [631]:
from datetime import timedelta  

In [632]:
asset_num = 50;

In [633]:
def get_first_timestamp():
    query_job = client.query("""SELECT Min(close_timestamp_ms) AS first_timestamp
FROM Binance_1m_BTC_klines.Binance_1m_BTC_1500005000_1548440000;""")
    first_timestamp = query_job.result()
    for row in first_timestamp:
        return row.first_timestamp

In [634]:
def get_last_timestamp():
    query_job = client.query("""SELECT Max(close_timestamp_ms) AS last_timestamp
FROM Binance_1m_BTC_klines.Binance_1m_BTC_1500005000_1548440000;""")
    last_timestamp = query_job.result()
    for row in last_timestamp:
        return row.last_timestamp

In [635]:
# todo minus day on last timestamp plus day on first timestamp

In [636]:
def get_top_assets_for_period(asset_num, start_time, end_time):
    top_symbols = []
    query_job = client.query("""
    SELECT base_asset, SUM(CAST(quote_asset_volume AS FLOAT64)) AS total_volume 
    FROM Binance_1m_BTC_klines.Binance_1m_BTC_1500005000_1548440000
    WHERE close_timestamp_ms>=1548346400000 and close_timestamp_ms<=1548440000000
    GROUP BY base_asset ORDER BY total_volume DESC LIMIT 50""")
    top_assets = query_job.result()
    for row in top_assets:
#         print("{} : {}".format(row.base_asset, row.total_volume))
        top_symbols.append(row.base_asset)
    return top_symbols;

In [637]:
# top_symbols = get_top_assets_for_period()

In [638]:
first_timestamp = get_first_timestamp()

In [639]:
last_timestamp = get_last_timestamp()

In [640]:
timestamp_range = last_timestamp - first_timestamp

In [641]:
first_timestamp = first_timestamp + (60000*1440)

In [642]:
end_timestamp = first_timestamp + (60000*1440)

In [643]:
last_timestamp = last_timestamp - (60000*1440)

In [644]:
start_timestamp = last_timestamp - (60000*1440)*50

In [645]:
1548353520000 - start_timestamp

4320000001

In [646]:
# todo round timestamp_ms

In [647]:
def get_features(start_time, end_time, selection_start_time, selection_end_time):
    query = """
    SELECT
    base_asset,
    timestamp_ms,
    CAST(close AS FLOAT64) AS c,
    CAST(open AS FLOAT64) AS o,
    CAST(low AS FLOAT64) AS l,
    CAST(high AS FLOAT64) AS h,
    CAST(quote_asset_volume AS FLOAT64) AS v,
    CAST(trades AS FLOAT64) AS t
    FROM Binance_1m_BTC_klines.Binance_1m_BTC_1500005000_1548440000
    WHERE base_asset IN (
        SELECT base_asset
        FROM Binance_1m_BTC_klines.Binance_1m_BTC_1500005000_1548440000
        WHERE close_timestamp_ms>={sts} AND close_timestamp_ms<={ets}
        GROUP BY base_asset ORDER BY SUM(CAST(quote_asset_volume AS FLOAT64)) DESC LIMIT 50
    )
    AND timestamp_ms>={st}
    AND timestamp_ms<={et}+1
    ORDER BY timestamp_ms;
    """.format(st=start_time, et=end_time, sts=selection_start_time, ets=selection_end_time)
    query_job = client.query(query)
    features = query_job.to_dataframe()
    return features

In [648]:
features = get_features(start_timestamp, last_timestamp, start_timestamp, last_timestamp)

In [649]:
x = features.copy()
x.set_index(['base_asset', 'timestamp_ms'], inplace=True)

In [650]:
x

,,c,o,l,h,v,t
base_asset,timestamp_ms,,,,,,
DASH,1544033520000,2.099200e-02,2.097000e-02,2.097000e-02,2.099200e-02,0.035133,5.0
REP,1544033520000,1.967000e-03,1.967000e-03,1.967000e-03,1.967000e-03,0.000000,0.0
DLT,1544033520000,1.055000e-05,1.055000e-05,1.055000e-05,1.055000e-05,0.008704,1.0
BCHSV,1544033520000,2.347400e-02,2.350000e-02,2.344300e-02,2.350000e-02,1.963974,20.0
ZRX,1544033520000,9.378000e-05,9.380000e-05,9.378000e-05,9.402000e-05,0.066807,6.0
NPXS,1544033520000,1.600000e-07,1.600000e-07,1.600000e-07,1.600000e-07,0.014346,1.0
VET,1544033520000,1.180000e-06,1.180000e-06,1.180000e-06,1.180000e-06,0.107774,4.0
BQX,1544033520000,2.972000e-05,2.972000e-05,2.972000e-05,2.973000e-05,0.108154,7.0
ARN,1544033520000,7.209000e-05,7.232000e-05,7.200000e-05,7.232000e-05,1.301251,22.0


In [651]:
t_index = list(range(int(start_timestamp/1000), int(last_timestamp/1000), 60))
time_index = pd.to_datetime(t_index,unit='s')
time_index = time_index.round('min');

In [652]:
y = features.copy()

In [653]:
y['timestamp_ms'] = pd.to_datetime(y['timestamp_ms'], unit='ms')
y.set_index(['base_asset', 'timestamp_ms'], inplace=True)
y=y.groupby(['base_asset','timestamp_ms']).first()
ind = pd.MultiIndex.from_product([y.index.levels[0], time_index], names=y.index.names)
y = y.reindex(ind)
y = y.fillna(axis=1, method="bfill").fillna(axis=1, method="ffill")

In [612]:
y.loc['ARN'].loc['2019-01-22 18:13:00'] = None

In [611]:
y.loc['ARN'].loc['2019-01-22 18:12:00'] = None

,c,o,l,h,v,t
timestamp_ms,,,,,,
2018-12-05 18:12:00,0.000072,0.000072,0.000072,0.000072,1.301251,22.0
2018-12-05 18:13:00,0.000072,0.000072,0.000072,0.000072,0.328146,8.0
2018-12-05 18:14:00,0.000072,0.000072,0.000072,0.000072,0.317190,8.0
2018-12-05 18:15:00,0.000072,0.000072,0.000072,0.000072,0.288332,11.0
2018-12-05 18:16:00,0.000072,0.000072,0.000072,0.000072,0.167546,5.0
2018-12-05 18:17:00,0.000072,0.000072,0.000072,0.000072,0.288985,12.0
2018-12-05 18:18:00,0.000072,0.000072,0.000072,0.000072,0.252518,4.0
2018-12-05 18:19:00,0.000072,0.000072,0.000072,0.000072,0.214500,8.0
2018-12-05 18:20:00,0.000072,0.000072,0.000072,0.000072,1.114984,11.0


In [615]:
y.fillna(axis=0, method="ffill")

c         o         l         h  \
base_asset timestamp_ms                                                  
ARN        2019-01-22 18:12:00       NaN       NaN       NaN       NaN   
           2019-01-22 18:13:00       NaN       NaN       NaN       NaN   
           2019-01-22 18:14:00  0.000080  0.000080  0.000080  0.000080   
           2019-01-22 18:15:00  0.000080  0.000080  0.000080  0.000080   
           2019-01-22 18:16:00  0.000080  0.000080  0.000080  0.000080   
           2019-01-22 18:17:00  0.000080  0.000080  0.000080  0.000080   
           2019-01-22 18:18:00  0.000080  0.000080  0.000080  0.000080   
           2019-01-22 18:19:00  0.000080  0.000080  0.000080  0.000080   
           2019-01-22 18:20:00  0.000080  0.000080  0.000080  0.000080   
           2019-01-22 18:21:00  0.000080  0.000080  0.000080  0.000080   
           2019-01-22 18:22:00  0.000080  0.000080  0.000080  0.000080   
           2019-01-22 18:23:00  0.000080  0.000080  0.000080  0.000080   
           2019-01-22 18:24:00  0.000080  0.000080  0.000080  0.000080   
           2019-01-22 18:25:00  0.000080  0.000080  0.000080  0.000080   
           2019-01-22 18:26:00  0.000080  0.000080  0.000080  0.000080   
           2019-01-22 18:27:00  0.000080  0.000080  0.000080  0.000080   
           2019-01-22 18:28:00  0.000080  0.000080  0.000080  0.000080   
           2019-01-22 18:29:00  0.000080  0.000080  0.000080  0.000080   
           2019-01-22 18:30:00  0.000080  0.000080  0.000080  0.000080   
           2019-01-22 18:31:00  0.000080  0.000080  0.000080  0.000080   
           2019-01-22 18:32:00  0.000080  0.000080  0.000080  0.000080   
           2019-01-22 18:33:00  0.000080  0.000080  0.000080  0.000080   
           2019-01-22 18:34:00  0.000080  0.000080  0.000080  0.000080   
           2019-01-22 18:35:00  0.000080  0.000080  0.000080  0.000080   
           2019-01-22 18:36:00  0.000080  0.000080  0.000080  0.000080   
           2019-01-22 18:37:00  0.000080  0.000080  0.000080  0.000080   
           2019-01-22 18:38:00  0.000080  0.000080  0.000080  0.000080   
           2019-01-22 18:39:00  0.000080  0.000080  0.000080  0.000080   
           2019-01-22 18:40:00  0.000080  0.000080  0.000080  0.000080   
           2019-01-22 18:41:00  0.000080  0.000080  0.000080  0.000080   
...                                  ...       ...       ...       ...   
ZRX        2019-01-24 17:42:00  0.000087  0.000087  0.000087  0.000087   
           2019-01-24 17:43:00  0.000087  0.000087  0.000086  0.000087   
           2019-01-24 17:44:00  0.000086  0.000087  0.000086  0.000087   
           2019-01-24 17:45:00  0.000086  0.000086  0.000086  0.000087   
           2019-01-24 17:46:00  0.000086  0.000086  0.000086  0.000086   
           2019-01-24 17:47:00  0.000087  0.000086  0.000086  0.000087   
           2019-01-24 17:48:00  0.000087  0.000086  0.000086  0.000087   
           2019-01-24 17:49:00  0.000087  0.000087  0.000087  0.000087   
           2019-01-24 17:50:00  0.000087  0.000087  0.000087  0.000087   
           2019-01-24 17:51:00  0.000087  0.000087  0.000087  0.000087   
           2019-01-24 17:52:00  0.000087  0.000087  0.000086  0.000087   
           2019-01-24 17:53:00  0.000087  0.000087  0.000087  0.000087   
           2019-01-24 17:54:00  0.000087  0.000087  0.000087  0.000087   
           2019-01-24 17:55:00  0.000087  0.000087  0.000087  0.000087   
           2019-01-24 17:56:00  0.000087  0.000087  0.000087  0.000087   
           2019-01-24 17:57:00  0.000087  0.000087  0.000087  0.000087   
           2019-01-24 17:58:00  0.000087  0.000087  0.000087  0.000087   
           2019-01-24 17:59:00  0.000087  0.000087  0.000087  0.000087   
           2019-01-24 18:00:00  0.000087  0.000087  0.000087  0.000087   
           2019-01-24 18:01:00  0.000087  0.000087  0.000087  0.000087   
           2019-01-24 18:02:00  0.000087  0.000087  0.000087  0.000087   
           2019-01-24 18:03:00  0.000087  0

In [588]:
m = y[['c','h','l']]

In [596]:
res = m.to_xarray()

In [597]:
arr = res.to_array()

In [600]:
arr[:, :, 50:50+90+1].shape

(3, 50, 91)

In [ ]:
# portfolio vector memory, [time, assets]
PVM = pd.DataFrame(
    index=y.index.levels[1],
    columns=y.index.levels[0]
)

In [ ]:
PVM = PVM.fillna(1.0 / len(y.index.levels[0]))